In [1]:
import os
import json
import spacy

from glob import glob
from tqdm import tqdm
from predpatt import PredPatt, load_comm
from predpatt.util.load import get_udparse
from concrete.util import CommunicationReader

In [2]:
def comm2udparses(comm, tool='Stanford CoreNLP basic'):
    if comm.sectionList:
        for sec in comm.sectionList:
            if sec.sentenceList:
                for sent in sec.sentenceList:
                    yield get_udparse(sent, tool)

In [3]:
cgw_path = '../data/cgw/raw'

## Sample a Concrete document from Gigaword

**Remember that all of that stuff is just to filter out documents, based on some list of keywords**

In [4]:
# TODO: (Anton) Get (pred, arg) tuples for each schema (for like 10 schemas)
# TODO: (Anton + Alexandra) Represent each doc as list of pairs (verb, argument)

# TODO: (Anton) index with Lucene (so that querying is fast)
# Schema(query) -> subset of docs (for each schema)

# HARD PART --> Implement and run IE stack on each subset of docs
# Run CD tools over IE processed docs,
#   where IE = {LOME (FrameNet), UDS, VerbNet, PredPatt?, ...}

for comm, filename in CommunicationReader(glob(f'{cgw_path}/*.tar.gz')[0]):
    # print(comm.text)
    for udparse in comm2udparses(comm):
        ppatt = PredPatt(udparse)
        print(' '.join([token.text for token in ppatt.tokens]))
        print(ppatt.pprint(color=True))
        print('Events (and positions):', [(x.root.text, x.root.position) for x in ppatt.events])
        print()
    break

WITNESS SAYS O.J. SIMPSON BOUGHT KNIFE WEEKS BEFORE SLAYINGS
	?a SAYS ?b
		?a: WITNESS
		?b: SOMETHING := O.J. SIMPSON BOUGHT KNIFE WEEKS BEFORE SLAYINGS
	?a WEEKS BEFORE SLAYINGS
		?a: O.J. SIMPSON BOUGHT KNIFE
Events (and positions): [('SAYS', 1), ('WEEKS', 6)]

LOS ANGELES -LRB- BC-SIMPSON-KILLINGS-1stLd-3Takes-Writethru-LADN -RRB-

Events (and positions): []

With the nation 's attention riveted again on a Los Angeles courtroom , a knife dealer testified that O.J. Simpson bought a 15-inch knife five weeks before the slashing deaths of his ex-wife and her friend .
	?a riveted again on a Los Angeles courtroom
		?a: the nation 's attention
	?a testified ?b
		?a: a knife dealer
		?b: SOMETHING := O.J. Simpson bought a 15-inch knife five weeks before the slashing deaths of his ex-wife and her friend
	?a bought ?b five weeks before the slashing deaths of his ex-wife and her friend
		?a: O.J. Simpson
		?b: a 15-inch knife
Events (and positions): [('riveted', 5), ('testified', 16), ('bough

---